In [1]:
import carla 
import math 
import random 
import time 
import numpy as np
import cv2
import open3d as o3d
from matplotlib import cm

# Connect the client and set up bp library and spawn point
client = carla.Client('localhost', 2000)
world = client.get_world()
bp_lib = world.get_blueprint_library() 
spawn_points = world.get_map().get_spawn_points() 

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [14]:
vehicle.destroy()

False

In [16]:
# Add vehicle
vehicle_bp = bp_lib.find('vehicle.mercedes-benz.coupe') 
vehicle_init_trans = carla.Transform(carla.Location(z=2, x=-80, y=-5))
vehicle = world.spawn_actor(vehicle_bp, vehicle_init_trans)
# vehicle.set_autopilot(True) 

In [18]:
# Move spectator to view ego vehicle
spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform)

In [4]:
# Auxilliary code for colormaps and axes
VIRIDIS = np.array(cm.get_cmap('plasma').colors)
VID_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])

COOL_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])
COOL = np.array(cm.get_cmap('winter')(COOL_RANGE))
COOL = COOL[:,:3]

def add_open3d_axis(vis):
    """Add a small 3D axis on Open3D Visualizer"""
    axis = o3d.geometry.LineSet()
    axis.points = o3d.utility.Vector3dVector(np.array([
        [0.0, 0.0, 0.0],
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]]))
    axis.lines = o3d.utility.Vector2iVector(np.array([
        [0, 1],
        [0, 2],
        [0, 3]]))
    axis.colors = o3d.utility.Vector3dVector(np.array([
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]]))
    vis.add_geometry(axis)

# LIDAR and RADAR callbacks
def lidar_callback(point_cloud, point_list):
    """Prepares a point cloud with intensity
    colors ready to be consumed by Open3D"""
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
    data = np.reshape(data, (int(data.shape[0] / 4), 4))

    # Isolate the intensity and compute a color for it
    intensity = data[:, -1]
    intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004 * 100))
    int_color = np.c_[
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 0]),
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 1]),
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 2])]

    points = data[:, :-1]

    points[:, :1] = -points[:, :1]

    point_list.points = o3d.utility.Vector3dVector(points)
    point_list.colors = o3d.utility.Vector3dVector(int_color)

def radar_callback(data, point_list):
    radar_data = np.zeros((len(data), 4))
    
    for i, detection in enumerate(data):
        x = detection.depth * math.cos(detection.altitude) * math.cos(detection.azimuth)
        y = detection.depth * math.cos(detection.altitude) * math.sin(detection.azimuth)
        z = detection.depth * math.sin(detection.altitude)
        
        radar_data[i, :] = [x, y, z, detection.velocity]
        
    intensity = np.abs(radar_data[:, -1])
    intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004 * 100))
    int_color = np.c_[
        np.interp(intensity_col, COOL_RANGE, COOL[:, 0]),
        np.interp(intensity_col, COOL_RANGE, COOL[:, 1]),
        np.interp(intensity_col, COOL_RANGE, COOL[:, 2])]
    
    points = radar_data[:, :-1]
    points[:, :1] = -points[:, :1]
    point_list.points = o3d.utility.Vector3dVector(points)
    point_list.colors = o3d.utility.Vector3dVector(int_color)
    
# Camera callback
def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

def sem_callback(image, data_dict):
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

def depth_callback(image, data_dict):
    image.convert(carla.ColorConverter.LogarithmicDepth)
    data_dict['depth_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    
def dvs_callback(data, data_dict):
    dvs_events = np.frombuffer(data.raw_data, dtype=np.dtype([
        ('x', np.uint16), ('y', np.uint16), ('t', np.int64), ('pol', np.bool)
    ]))
    data_dict['dvs_image'] = np.zeros((data.height, data.width, 4), dtype=np.uint8)
    dvs_img = np.zeros((data.height, data.width, 3), dtype=np.uint8)
    dvs_img[dvs_events[:]['y'], dvs_events[:]['x'], dvs_events[:]['pol']*2] = 255
    data_dict['dvs_image'][:, :, 0:3] = dvs_img

In [5]:
# Set up LIDAR and RADAR, parameters are to assisst visualisation
lidar_bp = bp_lib.find('sensor.lidar.ray_cast') 
lidar_bp.set_attribute('range', '100.0')
lidar_bp.set_attribute('noise_stddev', '0.1')
lidar_bp.set_attribute('upper_fov', '15.0')
lidar_bp.set_attribute('lower_fov', '-25.0')
lidar_bp.set_attribute('channels', '64.0')
lidar_bp.set_attribute('rotation_frequency', '20.0')
lidar_bp.set_attribute('points_per_second', '500000')
lidar_init_trans = carla.Transform(carla.Location(z=2))
lidar = world.spawn_actor(lidar_bp, lidar_init_trans, attach_to=vehicle)

radar_bp = bp_lib.find('sensor.other.radar') 
radar_bp.set_attribute('horizontal_fov', '30.0')
radar_bp.set_attribute('vertical_fov', '30.0')
radar_bp.set_attribute('points_per_second', '10000')
radar_init_trans = carla.Transform(carla.Location(z=2))
radar = world.spawn_actor(radar_bp, radar_init_trans, attach_to=vehicle)

# Add auxilliary data structures
point_list = o3d.geometry.PointCloud()
radar_list = o3d.geometry.PointCloud()

# Start sensors
lidar.listen(lambda data: lidar_callback(data, point_list))
radar.listen(lambda data: radar_callback(data, radar_list))

In [6]:
# all cameras
camera_init_trans = carla.Transform(carla.Location(z=2))

camera_bp = bp_lib.find('sensor.camera.rgb')
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

sem_camera_bp = bp_lib.find('sensor.camera.semantic_segmentation')
sem_camera = world.spawn_actor(sem_camera_bp, camera_init_trans, attach_to=vehicle)

dvs_camera_bp = bp_lib.find('sensor.camera.dvs')
dvs_camera = world.spawn_actor(dvs_camera_bp, camera_init_trans, attach_to=vehicle)

depth_camera_bp = bp_lib.find('sensor.camera.depth')
depth_camera = world.spawn_actor(depth_camera_bp, camera_init_trans, attach_to=vehicle)

image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()

sensor_data = {
    'image': np.zeros((image_h, image_w, 4)),
    'sem_image': np.zeros((image_h, image_w, 4)),
    'depth_image': np.zeros((image_h, image_w, 4)),
    'dvs_image': np.zeros((image_h, image_w, 4))
}

camera.listen(lambda image: camera_callback(image, sensor_data))
sem_camera.listen(lambda image: sem_callback(image, sensor_data))
depth_camera.listen(lambda image: depth_callback(image, sensor_data))
dvs_camera.listen(lambda image: dvs_callback(image, sensor_data))


In [7]:
# Cameras
cv2.namedWindow("All Cameras", cv2.WINDOW_AUTOSIZE)
top_row = np.concatenate((sensor_data['image'], sensor_data['sem_image']), axis=1)
bottom_row = np.concatenate((sensor_data['depth_image'], sensor_data['dvs_image']), axis=1)
tiled = np.concatenate((top_row, bottom_row), axis=0)

cv2.imshow("All Cameras", tiled)
cv2.waitKey(1)

# Open3D visualiser for LIDAR and RADAR
vis = o3d.visualization.Visualizer()
vis.create_window(
    window_name='Carla Lidar',
    width=960,
    height=540,
    left=480,
    top=270)
vis.get_render_option().background_color = [0.05, 0.05, 0.05]
vis.get_render_option().point_size = 1
vis.get_render_option().show_coordinate_frame = True
add_open3d_axis(vis)

# Update geometry and camera in game loop
frame = 0
while True:
    if frame == 2:
        vis.add_geometry(point_list)
        vis.add_geometry(radar_list)
    vis.update_geometry(point_list)
    vis.update_geometry(radar_list)
    
    vis.poll_events()
    vis.update_renderer()
    # # This can fix Open3D jittering issues:
    time.sleep(0.005)
    frame += 1

    top_row = np.concatenate((sensor_data['image'], sensor_data['sem_image']), axis=1)
    bottom_row = np.concatenate((sensor_data['depth_image'], sensor_data['dvs_image']), axis=1)
    tiled = np.concatenate((top_row, bottom_row), axis=0)
    cv2.imshow("All Cameras", tiled)
    
    # Break if user presses 'q'
    if cv2.waitKey(1) == ord('q'):
        break

# Close displayws and stop sensors
camera.stop()
sem_camera.stop()
depth_camera.stop()
dvs_camera.stop()
camera.destroy()
sem_camera.destroy()
depth_camera.destroy()
dvs_camera.destroy()
cv2.destroyAllWindows()

radar.stop()
radar.destroy()
lidar.stop()
lidar.destroy()
camera.stop()
camera.destroy()
vis.destroy_window()


QObject::moveToThread: Current thread (0x55610d1b2b00) is not the object's thread (0x55610edb49d0).
Cannot move to target thread (0x55610d1b2b00)

QObject::moveToThread: Current thread (0x55610d1b2b00) is not the object's thread (0x55610edb49d0).
Cannot move to target thread (0x55610d1b2b00)

QObject::moveToThread: Current thread (0x55610d1b2b00) is not the object's thread (0x55610edb49d0).
Cannot move to target thread (0x55610d1b2b00)

QObject::moveToThread: Current thread (0x55610d1b2b00) is not the object's thread (0x55610edb49d0).
Cannot move to target thread (0x55610d1b2b00)

QObject::moveToThread: Current thread (0x55610d1b2b00) is not the object's thread (0x55610edb49d0).
Cannot move to target thread (0x55610d1b2b00)

QObject::moveToThread: Current thread (0x55610d1b2b00) is not the object's thread (0x55610edb49d0).
Cannot move to target thread (0x55610d1b2b00)

QObject::moveToThread: Current thread (0x55610d1b2b00) is not the object's thread (0x55610edb49d0).
Cannot move to tar

/tmp/ipykernel_17269/812631185.py:85: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ('x', np.uint16), ('y', np.uint16), ('t', np.int64), ('pol', np.bool)
/tmp/ipykernel_17269/812631185.py:60: RuntimeWarning: divide by zero encountered in log
  intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004 * 100))
